In [79]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.headless = True

import numpy as np
import pandas as pd
import time as time
from time import sleep
import random
from tqdm import tqdm


In [13]:
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.library.parameters import SeasonAll

In [120]:
seasons = ['200{}-0{}'.format(x, x+1) if x != 9 else '200{}-{}'.format(x, x+1) for x in range(6, 10)]
seasons2 = ['20{}-{}'.format(x, x+1) for x in range(10, 21)]

seasons.extend(seasons2)
dates = []
for season in seasons:
    date = []
    for season_type in ['Regular Season', 'Playoffs']:
        games = leaguegamelog.LeagueGameLog(season=season, season_type_all_star=season_type).get_data_frames()[0]
        dates.extend(games['GAME_DATE'].unique())
    sleep(2)

dates[-10:]

['2021-02-22',
 '2021-02-23',
 '2021-02-24',
 '2021-02-25',
 '2021-02-26',
 '2021-02-27',
 '2021-02-28',
 '2021-03-01',
 '2021-03-02',
 '2021-03-03']

In [127]:
current_dates = pd.read_csv('../data/all_moneylines.csv')
current_dates = current_dates['game_date'].unique()

dates_to_add = set(dates) - set(current_dates)

print(len(dates_to_add))
print(list(dates_to_add)[:20])


451
['2017-05-11', '2012-06-12', '2014-06-15', '2013-06-18', '2020-09-02', '2015-05-26', '2013-04-21', '2020-09-25', '2014-04-27', '2016-05-28', '2011-02-12', '2015-04-22', '2017-06-04', '2013-05-11', '2011-04-26', '2014-05-07', '2017-04-19', '2019-04-25', '2012-05-27', '2020-09-15']


In [155]:
# Get Moneylines

gm_date1 = []
away_teams1 = []
home_teams1 = []
away_scoreboards1 = []
home_scoreboards1 = []
away_moneylines1 = []
home_moneylines1 = []

for date in tqdm(['2021-03-02', '2021-03-03'], desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams1.append(away_team)
        home_teams1.append(home_team)
        gm_date1.append(date)

#         scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

#         home_score = []
#         away_score = []

#         for score in scoreboard:
#             quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
#             for i in range(len(quarters)):
#                 scores = quarters[i].text.split('\n')
#                 away_score.append(scores[0])
#                 home_score.append(scores[1])
#             away_scoreboards1.append(away_score)
#             home_scoreboards1.append(home_score)

        mls = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(mls)):    
            if i % 2 == 0:
                away_lines.append(mls[i].text)
            else:
                home_lines.append(mls[i].text)
        away_moneylines1.append(away_lines)
        home_moneylines1.append(home_lines)

    driver.quit()
    sleep(random.randint(1,2))



progress: 100%|██████████████████████████████████████████████████████████████████████████| 2/2 [00:23<00:00, 11.82s/it]


In [159]:
print(len(away_teams1))
print(len(home_teams1))
print(len(gm_date1))
print(len(away_scoreboards1))
print(len(home_scoreboards1))
print(len(away_moneylines1))
print(len(home_moneylines1))

df1 = pd.read_csv('../data/all_moneylines_sbr.csv')
# df1
df2 = pd.DataFrame({'away_team':away_teams1,
                  'home_team':home_teams1,
                   'game_date':gm_date1,
                  'away_moneyline':away_moneylines1,
                  'home_moneyline':home_moneylines1})


updated_mls = pd.concat([df1, df2])
updated_mls = updated_mls.drop_duplicates(subset=['away_team', 'home_team', 'game_date'], keep='last')
updated_mls.tail(20)

updated_mls.to_csv("../data/all_moneylines_sbr.csv", index=False)


16
16
16
0
0
16
16


In [104]:
df2

,away_team,home_team,game_date,away_moneyline,home_moneyline
0,L.A. Lakers,Charlotte,2011-02-13,"[-225, -220, -215, -]","[+185, +180, +185, -]"
1,Sacramento,Oklahoma City,2011-02-13,"[+375, +350, +400, -]","[-550, -500, -500, -]"
2,Miami,Indiana,2011-02-13,"[-240, -230, -245, -]","[+200, +190, +205, -]"
3,Charlotte,Chicago,2011-02-13,"[+400, +400, +450, -]","[-600, -600, -550, -]"
4,Atlanta,Detroit,2011-02-13,"[-, -175, -190, -]","[-, +155, +160, -]"
...,...,...,...,...,...
23853,Atlanta,Miami,2021-02-28,"[+120, +120, +125, +120]","[-140, -140, -145, -140]"
23854,Memphis,Houston,2021-02-28,"[-165, -170, -163, -165]","[+145, +150, +142, +145]"
23855,Golden State,L.A. Lakers,2021-02-28,"[-220, +125, +122, +125]","[+160, -145, -143, -145]"
23856,Phoenix,Minnesota,2021-02-28,"[-600, -530, -594, -560]","[+400, +425, +443, +435]"


In [89]:
df1 = df1.drop(columns=['away_scoreboard', 'home_scoreboard'])

In [91]:
all_moneylines = pd.concat([df1, df2])
all_moneylines.to_csv('../data/all_moneylines.csv', index=False)


In [94]:
set(dates) - set(all_moneylines['game_date'])

{'2011-02-12', '2011-02-15', '2011-02-16'}

In [98]:
dates[0]

'2010-10-26'

In [133]:
# Get Spreads

current_dates = pd.read_csv('../data/all_spreads_sbr.csv')
current_dates = current_dates['game_date'].unique()

dates_to_add = set(dates) - set(current_dates)

print("adding:", len(dates_to_add), "dates")

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_spreads = []
home_spreads = []

for date in tqdm(dates_to_add, desc='progress'):
    web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={}'.format(date)
    path = '../chromedriver.exe'
    driver = webdriver.Chrome(path)
    driver.get(web)

    single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

    num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

    num_listed_events = len(single_row_events)
    cutoff = num_listed_events - num_postponed_events

    for event in single_row_events[:cutoff]:

        away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
        home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
        away_teams.append(away_team)
        home_teams.append(home_team)
        gm_date.append(date)


        scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

        home_score = []
        away_score = []

        for score in scoreboard:
            quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
            for i in range(len(quarters)):
                scores = quarters[i].text.split('\n')
                away_score.append(scores[0])
                home_score.append(scores[1])
            away_scoreboards.append(away_score)
            home_scoreboards.append(home_score)
            
            
        if len(away_scoreboards) != len(away_teams):
            num_to_add = len(away_teams) - len(away_scoreboards)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])
            
        
        spreads = event.find_elements_by_class_name('pointer-2j4Dk')
        away_lines = []
        home_lines = []
        for i in range(len(spreads)):    
            if i % 2 == 0:
                away_lines.append(spreads[i].text)
            else:
                home_lines.append(spreads[i].text)
        away_spreads.append(away_lines)
        home_spreads.append(home_lines)
        
        if len(away_spreads) != len(away_teams):
            num_to_add = len(away_teams) - len(away_spreads)
            for i in range(num_to_add):
                away_scoreboards.append([])
                home_scoreboards.append([])
            
        
    driver.quit()
    sleep(random.randint(1,2))

spread_df = pd.DataFrame({'away_team':away_teams,
                  'home_team':home_teams,
                   'game_date':gm_date,
                  'away_scoreboard':away_scoreboards,
                  'home_scoreboard':home_scoreboards,
                  'away_spreads':away_spreads,
                  'home_spreads':home_spreads})

spread_df


progress:   0%|                                                                                | 0/627 [00:00<?, ?it/s]

adding: 627 dates


progress: 100%|████████████████████████████████████████████████████████████████████| 627/627 [2:27:34<00:00, 14.12s/it]


,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_moneyline,home_moneyline
0,San Antonio,Houston,2017-05-11,"[31, 30, 26, 27, 114]","[24, 18, 22, 11, 75]","[+9-115, +9-108, +8½-110, +9-105]","[-9-105, -9-112, -8½-110, -9-105]"
1,Atlanta,Boston,2008-05-04,"[16, 10, 17, 22, 65]","[27, 17, 35, 20, 99]","[+14½-105, +15-110, +15-110, -]","[-14½-115, -15-110, -15-110, -]"
2,Utah,L.A. Lakers,2008-05-04,"[24, 17, 31, 26, 98]","[25, 29, 25, 30, 109]","[+8½-110, +8½-105, +8½-110, -]","[-8½-110, -8½-115, -8½-110, -]"
3,L.A. Lakers,Houston,2009-05-10,"[16, 20, 18, 33, 87]","[29, 25, 29, 16, 99]",[],[]
4,Boston,Orlando,2009-05-10,"[25, 23, 31, 16, 95]","[28, 18, 25, 23, 94]","[+5-115, +5½-110, +5½-110, -]","[-5-105, -5½-110, -5½-110, -]"
...,...,...,...,...,...,...,...
1167,Phoenix,San Antonio,2007-05-12,"[30, 23, 19, 29, 101]","[25, 30, 25, 28, 108]","[+4-110, +4½-112, +4-110, -]","[-4-110, -4½-108, -4-110, -]"
1168,Toronto,Orlando,2008-04-28,"[26, 24, 18, 24, 92]","[22, 27, 25, 28, 102]","[+7½-105, +7½-110, +7-110, -]","[-7½-115, -7½-110, -7-110, -]"
1169,Boston,Atlanta,2008-04-28,"[24, 24, 27, 17, 92]","[29, 22, 14, 32, 97]","[-10-105, -9-110, -9-110, -]","[+10-115, +9-110, +9-110, -]"
1170,L.A. Lakers,Denver,2008-04-28,"[32, 32, 15, 28, 107]","[23, 31, 23, 24, 101]","[-5-105, -4-110, -4-110, -]","[+5-115, +4-110, +4-110, -]"


In [135]:
spreads_orig = pd.read_csv("../data/all_spreads_sbr.csv")

updated_spreads = pd.concat([spreads_orig, spread_df])

updated_spreads['game_date'] = pd.to_datetime(updated_spreads['game_date'])
updated_spreads.sort_values(['game_date'], inplace=True)

updated_spreads.rename(columns={'away_moneyline':'away_spread',
                               'home_moneyline':'home_spread'}, inplace=True)
updated_spreads

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_spread,home_spread
0,Chicago,Miami,2006-10-31,"['22', '37', '21', '28', '108']","['16', '14', '21', '15', '66']","['+4½-110', '-', '+4½-110', '-']","['-4½-110', '-', '-4½-110', '-']"
1129,Phoenix,L.A. Lakers,2006-10-31,"['41', '17', '21', '27', '106']","['26', '27', '34', '27', '114']","['-8½-110', '-', '-8-110', '-']","['+8½-110', '-', '+8-110', '-']"
1128,Chicago,Miami,2006-10-31,"['22', '37', '21', '28', '108']","['16', '14', '21', '15', '66']","['+4½-110', '-', '+4½-110', '-']","['-4½-110', '-', '-4½-110', '-']"
1,Phoenix,L.A. Lakers,2006-10-31,"['41', '17', '21', '27', '106']","['26', '27', '34', '27', '114']","['-8½-110', '-', '-8-110', '-']","['+8½-110', '-', '+8-110', '-']"
3,Chicago,Orlando,2006-11-01,"['20', '25', '25', '24', '94']","['32', '31', '23', '23', '109']","['-2-110', '-', '-1-110', '-']","['+2-110', '-', '+1-110', '-']"
...,...,...,...,...,...,...,...
501,Atlanta,Orlando,2021-03-03,"[27, 15, 36, 37, 115]","[39, 22, 29, 22, 112]","[-3-105, -3-107, -2½-110, -2½-108]","[+3-115, +3-113, +2½-110, +2½-108]"
500,Brooklyn,Houston,2021-03-03,"[30, 37, 34, 31, 132]","[27, 27, 31, 29, 114]","[-10½-115, -10½-110, -11-110, -11-108]","[+10½-105, +10½-110, +11-110, +11-108]"
499,Detroit,Toronto,2021-03-03,"[43, 26, 29, 31, 129]","[37, 23, 20, 25, 105]","[+7½-110, +7½-110, +7½-105, +8-108]","[-7½-110, -7½-110, -7½-115, -8-108]"
506,Golden State,Portland,2021-03-03,"[29, 27, 24, 26, 106]","[28, 27, 27, 26, 108]","[-5½-120, -1-115, -1-115, -1-108]","[+5½-120, +1-105, +1-105, +1-108]"


In [138]:
updated_spreads = updated_spreads.drop_duplicates(['away_team','home_team', 'game_date'])

In [139]:
updated_spreads.to_csv("../data/all_spreads_sbr.csv", index=False)

In [114]:
spread_df

,away_team,home_team,game_date,away_scoreboard,home_scoreboard,away_moneyline,home_moneyline
0,Chicago,Miami,2006-10-31,"[22, 37, 21, 28, 108]","[16, 14, 21, 15, 66]","[+4½-110, -, +4½-110, -]","[-4½-110, -, -4½-110, -]"
1,Phoenix,L.A. Lakers,2006-10-31,"[41, 17, 21, 27, 106]","[26, 27, 34, 27, 114]","[-8½-110, -, -8-110, -]","[+8½-110, -, +8-110, -]"
2,Indiana,Charlotte,2006-11-01,"[23, 26, 28, 29, 106]","[27, 23, 23, 26, 99]","[-1-110, -, -1-110, -]","[+1-110, -, +1-110, -]"
3,Chicago,Orlando,2006-11-01,"[20, 25, 25, 24, 94]","[32, 31, 23, 23, 109]","[-2-110, -, -1-110, -]","[+2-110, -, +1-110, -]"
4,Atlanta,Philadelphia,2006-11-01,"[14, 21, 23, 17, 75]","[29, 20, 24, 15, 88]","[+6-115, -, +5½-110, -]","[-6-105, -, -5½-110, -]"
...,...,...,...,...,...,...,...
33579,Atlanta,Miami,2021-03-02,"[23, 21, 19, 31, 94]","[17, 20, 29, 14, 80]","[+2-110, +2½-115, +2½-110, +2-108]","[-2-110, -2½-105, -2½-110, -2-108]"
33580,L.A. Clippers,Boston,2021-03-02,"[32, 31, 26, 23, 112]","[35, 27, 26, 29, 117]","[+8½-120, -5-110, -3-110, -5-108]","[-8½-120, +5-110, +3-110, +5-108]"
33581,New York,San Antonio,2021-03-02,"[23, 24, 21, 25, 93]","[25, 26, 36, 32, 119]","[-2-105, -1½-113, -1½-105, -2-108]","[+2-115, +1½-107, +1½-115, +2-108]"
33582,Denver,Milwaukee,2021-03-02,"[37, 27, 35, 29, 128]","[23, 27, 30, 17, 97]","[+8-110, +8-109, +7½-105, +8-108]","[-8-110, -8-111, -7½-115, -8-108]"


In [115]:
spread_df.rename(columns={'away_moneyline':'away_spread', 'home_moneyline':'home_spread'})
spread_df.to_csv("../data/all_spreads_sbr.csv", index=False)

In [82]:

gm_date = []
away_teams = []
home_teams = []
away_scoreboards = []
home_scoreboards = []
away_moneylines = []
home_moneylines = []

web = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={}'.format(date)
path = '../chromedriver.exe'
driver = webdriver.Chrome(path)
driver.get(web)

single_row_events = driver.find_elements_by_class_name('eventMarketGridContainer-3QipG')

num_postponed_events = len(driver.find_elements_by_class_name('eventStatus-3EHqw'))

num_listed_events = len(single_row_events)
cutoff = num_listed_events - num_postponed_events

for event in single_row_events[:cutoff]:

    away_team = event.find_elements_by_class_name('participantBox-3ar9Y')[0].text
    home_team = event.find_elements_by_class_name('participantBox-3ar9Y')[1].text
    away_teams.append(away_team)
    home_teams.append(home_team)
    gm_date.append(date)


    scoreboard = event.find_elements_by_class_name('scoreboard-1TXQV')

    home_score = []
    away_score = []

    for score in scoreboard:
        quarters = score.find_elements_by_class_name('scoreboardColumn-2OtpR')
        for i in range(len(quarters)):
            scores = quarters[i].text.split('\n')
            away_score.append(scores[0])
            home_score.append(scores[1])

        away_scoreboards.append(away_score)
        home_scoreboards.append(home_score)

    mls = event.find_elements_by_class_name('pointer-2j4Dk')
    away_lines = []
    home_lines = []
    for i in range(len(mls)):    
        if i % 2 == 0:
            away_lines.append(mls[i].text)
        else:
            home_lines.append(mls[i].text)
    away_moneylines.append(away_lines)
    home_moneylines.append(home_lines)

    driver.quit()